# Training with Cached Features
With this option, we preform feature extraction as a separate step prior to training and save the features so they can be used in training. This improves training runtime.

## Feature Extraction
You can run feature extraction as a separate step, prior to training. This will make training run faster, but is very expensive in terms of disk space and time, as there are not any built in options inside of the OpenGlue framework to preform feature extraction on a subset of the data. The only ways to do this are to either manually move all of the scene folders not included in the files specifying the subsets generated in the previous step to a different location, or to not run feature extraction as a separate step and instead run OpenGlue training such that feature extraction is done concurrently with training, meaning only the scene files used for training/validation will have feature extraction preformed on them.
<br />The steps to do feature extraction as a separate step are detailed in the README:<br />
run `python extract_features.py` with the following parameters (for more details, please, refer to module's documentation):
      
   * `--device` - `[cpu, cuda]`
   * `--num_workers` - number of workers for parallel processing. When cpu is chosen, assigns the exact amount of the set workers, for gpu scenario it takes into account the number of gpu units available. 
   * `--target_size` - target size of the image (WIDTH, HEIGHT).
   * `--data_path` - path to directory with scenes images
   * `--output_path` - path to directory where extracted features are stored
   * `--extractor_config_path` - 'path to the file containing config for feature extractor in .yaml format
   * `--recompute` - include this flag to recompute features if it is already present in output directory
   * `--image_format` - formats of images searched inside `data_path` to compute features for, default: [jpg, JPEG, JPG, png]
   
   Choosing local feature extractor is performed via `--extractor_config_path`. 
   We provide configs in `config/features/`, which user can edit or use unchanged:
   * SuperPoint with MagicLeap weights - [`superpoint_magicleap.yaml`](config/features/superpoint_magicleap.yaml)
   * SuperPoint with KITTI weights - [`superpoint_kitti.yaml`](config/features/superpoint_kitti.yaml)
   * SuperPoint with COCO weights - [`superpoint_coco.yaml`](config/features/superpoint_coco.yaml)
   * SIFT opencv - [`sift_opencv.yaml`](config/features/sift_opencv.yaml)
   * DoG-AffNet-HardNet - [`dog_opencv_affnet_hardnet.yaml`](config/features/dog_opencv_affnet_hardnet.yaml) Note: I would reccommend this not be used
<br />I have included an example command to run this from the command line, as well as reimplemented the necessary functions such that it can be run from here, with the arguments being in a list rather than entered as command line arguments. The command line version would look like:<br /> `python extract_features.py --device 'cuda' --num_workers '1' --target_size 1280 1280 --data_path '/host_Data/Data/MegaDepth/MegaDepth/phoenix/S6/zl548/MegaDepth_v1/' --output_path '/host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix' --extractor_config_path 'config/features/superpoint_magicleap.yaml'`<br />
Note: if feature extraction is run as a separate step, be sure to take note of the target size you use. In this case it is 1280 by 1280, which is what I used when I ran feature extraction on it's own, but this value needs to be consistent inside the config files to be used during training. Also, this value could be made smaller, which may improve runtimes.

#### Selecting a Feature Extractor

To choose a specific feature extractor for pre-extraction or realtime extraction, you specify the filepath to the config file for that feature extractor, containing neccessary configurations and a filepath to the model weights if applicable. 3 of these extractors, the superpoint extractors, use pretrained models, while the others do not. It is reccommended to use one of these 3: superpoint_magicleap, superpoint_kitti, or superpoint_coco. These are differentiated by the dataset that was used to train them. 2 sets of default extractors are included, one in config/features/ and one in config/features_online/<br />
In config/features/, the extractors are configured to run with a higher number of maximum keypoints. For the superpoint extractors, this is set at 2048 keypoints. In features online, the extractors are configured to run with a lower number of maximum keypoints, which helps reduce runtime and memory utilization. For the superpoint extractors, this is 1024 keypoints. <br/><br/>
For training with cached-features, you must also specify the maximum number of keypoints in the configuration file used for training. This should be less than or equal to the number used in feature extraction, though it is reccommended that they be the same value.

In [1]:
#From: extract_features.py
'''
Modified to not use argparse in interactive environment and to allow for only preforming feature extraction on a subset of the data
'''

import glob
import logging
import math
import pathlib
from pathlib import Path
from typing import Tuple, List, Union, Optional

import cv2
import deepdish as dd
import numpy as np
import os
import torch
import yaml
from torch import multiprocessing

from models.features import get_feature_extractor

from multiprocessing_helper import process_chunk
import extract_features

def get_images_list(input_data_path: pathlib.Path, image_formats: List[str], scene_subset_path: pathlib.Path) -> List[Tuple[str, Optional[str]]]:
    """
    Get list of images that wil be processed.
    Args:
        input_data_path: input path to the location with images
        image_formats: file formats of images to look for
        scence_subset_path: input path to file specifying which scenes to use. All scenes used if set to ./None
    Returns:
        images_path_list: list where each image is represented as tuple with path to image
        and scene name (or None if no scenes are available)
    """
    
    scene_set = []
    if not scene_subset_path.match("./None"):
        with open(scene_subset_path, 'r') as fin:
            scene_set = fin.read().split('\n')
    print(scene_set)
    # process each scene
    images_path_list = []
    
    #Added to take subset prior to extraction
    scenes = os.listdir(input_data_path)
    for scene in scenes:
        if scene not in scene_set and len(scene_set) > 0:
            continue
            
            
        images_path = input_data_path / scene / 'dense0' / 'imgs'
        scene_list = []
        for image_format in image_formats:
            scene_list.extend(glob.glob(str(images_path / f'*.{image_format}')))
        images_path_list.extend(((path, scene) for path in scene_list))
    return images_path_list

def get_output_directory_name(feature_extractor_config: dict, target_size) -> str:
    """
    Build output directory name based on parameters
    Args:
        feature_extractor_config: parameters of feature extractor
        args: command line arguments

    Returns:
        Name of directory where features are stored
    """
    name = feature_extractor_config['name']
    if target_size is not None:
        name += f'_{target_size[0]}_{target_size[1]}'
    return name


def extract(data_path, output_path, device='cpu', num_workers=1, target_size=None, extractor_config_path=Path('config/features/sift_opencv.yaml'), image_format=['jpg', 'JPEG', 'JPG', 'png'], scene_subset_path=Path('./None'), recompute=False):
    logger = logging.getLogger(__name__)

    if device == 'cuda' and torch.cuda.device_count() < num_workers:
        logger.warning(f'Number of workers selected is bigger than number of available cuda devices. '
                       f'Setting num_workers to {torch.cuda.device_count()}.')
        num_workers = torch.cuda.device_count()

    # read feature extractor config
    with open(extractor_config_path) as f:
        feature_extractor_config = yaml.full_load(f)

    # make output directory
    output_path = output_path / get_output_directory_name(feature_extractor_config, target_size)
    logger.info(f'Creating output directory {output_path} (if not exists).')
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, 'config.yaml'), 'w') as f:
        yaml.dump(feature_extractor_config, f)


    #modified to add scene_subset_path to preform extraction on only a subset of data specified in the file at scene_subset_path
    images_list = get_images_list(data_path, image_format, scene_subset_path)
    logger.info(f'Total number of images found to process: {len(images_list)}')
    # split into chunks of (almost) equal size
    chunk_size = math.ceil(len(images_list) / num_workers)
    images_list = [images_list[i * chunk_size:(i + 1) * chunk_size] for i in range(num_workers)]

    logger.info(f'Starting {num_workers} processes for features extraction.')
    multiprocessing.start_processes(
        process_chunk,
        args=(images_list, feature_extractor_config, output_path, device, recompute, target_size),
        nprocs=num_workers,
        join=True
    )

/root/code/OpenGlue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
device = 'cuda'
num_workers = 1
target_size = [960, 720]
data_path = Path('/host_Data/Data/MegaDepth/MegaDepth/phoenix/S6/zl548/MegaDepth_v1/')
output_path = Path('/host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix3')
extractor_config_path = Path('./config/features_online/superpoint_magicleap.yaml')
scene_subset_path = Path('./assets/subset-20-20-total.txt')
#scene_subset_path = Path('./None') #Use to preform extraction on the entire dataset
extract(device=device, num_workers=num_workers, target_size=target_size, data_path=data_path, output_path=output_path, extractor_config_path=extractor_config_path, scene_subset_path=scene_subset_path)


[2022/06/15 07:16:32] __main__ | INFO: Creating output directory /host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix3/SuperPointNet_960_720 (if not exists).
[2022/06/15 07:16:32] __main__ | INFO: Total number of images found to process: 2147
[2022/06/15 07:16:32] __main__ | INFO: Starting 1 processes for features extraction.


['0024', '0048', '0102', '0148', '0181', '0307', '']
<All keys matched successfully>


#### Configuration Options

See [`CONFIGURATIONS.md`](CONFIGURATIONS.md) for configuration details. Please ensure all config options are set properly prior to training. For pre-extraction (cached features), config/config_cached.yaml will be used as default, but if you specify a different config file in the arguments, then it will be merged with config/config_cached.yaml, overwritting such that the settings in the specified config file are kept over the conflicting ones in config/config_cached.yaml. So, modify config_cached.yaml or create your own to use with the proper config options set.

## Training with pre-extraction (cached)

In [1]:
#from train_cached.py

import torch
import shutup

shutup.please()
import os
import argparse
from datetime import datetime
from omegaconf import OmegaConf
import pytorch_lightning as pl
from pytorch_lightning.strategies import DataParallelStrategy

from data.megadepth_datamodule import MegaDepthPairsDataModuleFeatures
from models.matching_module import MatchingTrainingModule
from utils.train_utils import get_training_loggers, get_training_callbacks, prepare_logging_directory


def train_cached(config_path='config/config_cached.yaml'):

    # Load config
    config = OmegaConf.load('config/config_cached.yaml')  # base config
    if config_path != 'config/config_cached.yaml':
        add_conf = OmegaConf.load(config_path)
        config = OmegaConf.merge(config, add_conf)

    pl.seed_everything(int(os.environ.get('LOCAL_RANK', 0)))
    
    # moved assignment of features_config before experiment_name creation to facilitate correcting error inputting
    # the entire path to the extracted features being used as part of the experiment name
    features_config = OmegaConf.load(os.path.join(config['data']['root_path'],
                                               config['data']['features_dir'], 'config.yaml'))

    # Prepare directory for logs and checkpoints
    # Use features_config['name'] rather than config['data']['features_dir'] to prevent entire path
    # from being inputted as part of the experiment name, resulting in logs being saved in an unexpected location
    if os.environ.get('LOCAL_RANK', 0) == 0:
        experiment_name = '{}_cache__attn_{}__laf_{}__{}'.format(
            features_config['name'],
            config['superglue']['attention_gnn']['attention'],
            config['superglue']['laf_to_sideinfo_method'],
            str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        )
        log_path = prepare_logging_directory(config, experiment_name)
    else:
        experiment_name, log_path = '', ''
    print(experiment_name, log_path)

    # Init Lightning Data Module
    data_config = config['data']
    dm = MegaDepthPairsDataModuleFeatures(
        root_path=data_config['root_path'],
        train_list_path=data_config['train_list_path'],
        val_list_path=data_config['val_list_path'],
        test_list_path=data_config['test_list_path'],
        batch_size=data_config['batch_size_per_gpu'],
        num_workers=data_config['dataloader_workers_per_gpu'],
        target_size=data_config['target_size'],
        features_dir=data_config['features_dir'],
        num_keypoints=data_config['max_keypoints'],
        val_max_pairs_per_scene=data_config['val_max_pairs_per_scene'],
        balanced_train=data_config.get('balanced_train', False),
        train_pairs_overlap=data_config.get('train_pairs_overlap')
    )


    # Init model
    model = MatchingTrainingModule(
        train_config={**config['train'], **config['inference'], **config['evaluation']},
        features_config=features_config,
        superglue_config=config['superglue'],
    )

    # Set callbacks and loggers
    callbacks = get_training_callbacks(config, log_path, experiment_name)
    loggers = get_training_loggers(config, log_path, experiment_name)

    # Init trainer
    trainer = pl.Trainer(
        gpus=config['gpus'],
        max_epochs=config['train']['epochs'],
        accelerator="gpu",
        gradient_clip_val=config['train']['grad_clip'],
        log_every_n_steps=config['logging']['train_logs_steps'],
        limit_train_batches=config['train']['steps_per_epoch'],
        num_sanity_val_steps=5,
        callbacks=callbacks,
        logger=loggers,
        strategy=DataParallelStrategy(),
        #plugins=DDPPlugin(find_unused_parameters=False),
        precision=config['train'].get('precision', 32),
    )
    # If loaded from checkpoint - validate
    if config.get('checkpoint') is not None:
        trainer.validate(model, datamodule=dm, ckpt_path=config.get('checkpoint'))
    trainer.fit(model, datamodule=dm, ckpt_path=config.get('checkpoint'))


/root/code/OpenGlue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Ensure torch in interactive python is working properly
Sometimes, torch running inside of an interactive python environment fails to recognize cuda devices. If you have a cuda device you expect to be detected, run this to make sure that torch finds it. If this does not display the expected number of devices, especially if it detects none, try restarting the container that the interactive environment is running on.

In [7]:
print('Number of devices found: ', torch.cuda.device_count())

Number of devices found:  1



Set the path to the config file you would like to use for training with cached features and begin training in the interactive environment below. config/config_cached.yaml is used by default <br />Alternatively,
<b>To launch train as a script with cached features, run: </b>  
```
python train_cached.py --config='config/config_cached.yaml'
```
This will utilize DDPStrategy (Distributed Data Parallel), as opposed to DataParallelStrategy. In the interactive environment, DDPStrategy is incompatible, so DataParallelStrategy is used instead.

In [3]:
config = 'config/config_cached.yaml'
train_cached(config)

Global seed set to 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


logs SuperPointNet_960_720_preextracted SuperPointNet_cache__attn_softmax__laf_none__2022-06-15-17-11-06
log path logs/SuperPointNet_960_720_preextracted/SuperPointNet_cache__attn_softmax__laf_none__2022-06-15-17-11-06
SuperPointNet_cache__attn_softmax__laf_none__2022-06-15-17-11-06 logs/SuperPointNet_960_720_preextracted/SuperPointNet_cache__attn_softmax__laf_none__2022-06-15-17-11-06


wandb: Currently logged in as: jlunder. Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                      | Params
---------------------------------------------------------------------
0 | superglue              | SuperGlue                 | 12.0 M
1 | augmentations          | AugmentationSequential    | 0     
2 | epipolar_dist_metric   | AccuracyUsingEpipolarDist | 0     
3 | camera_pose_auc_metric | CameraPoseAUC             | 0     
---------------------------------------------------------------------
12.0 M    Trainable params
0         Non-trainable params
12.0 M    Total params
47.829    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]